<a href="https://colab.research.google.com/github/Shtepser/Manyinterpret/blob/master/Day2/%D0%A2%D1%8E%D0%BC%D0%B5%D0%BD%D1%8C_BERT_%D1%8F%D0%B7%D1%8B%D0%BA%D0%BE%D0%B2%D0%B0%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

В современной компьютерной лингвистике вычисление вероятности текста производится в основном за счёт нейронных, а не энграммных моделей. Существует много разновидностей архитектур, мы языковую модель с пропусками (MLM) `BERT`.

## Моделирование с пропусками: BERT

In [ ]:
from transformers import AutoTokenizer, BertForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = BertForMaskedLM.from_pretrained("bert-base-cased").to("cuda")

В модели БЕРТ часть слов можно заменять масками. Модель предсказывает распределение вероятности для каждой позиции (в том числе для слов, присутствующих в тексте).

In [ ]:
 sentences = [
     "Yesterday, all my troubles seemed so far away.",
     "Yesterday, all my [MASK] seemed so far away.",
     "Several space rockets work on dymetylhydrasin.",
     "The Starship prototype descended under active aerodynamic control, accomplished by four vehicles.",
     "Спасибо, кончено — прощай, Москва!"
 ]
 tokenization = tokenizer(sentences)
 for sentence in tokenization["input_ids"]:
     print(*sentence)
     print(tokenizer.convert_ids_to_tokens(sentence))
     print()

In [ ]:
tokenizer.special_tokens_map

Напишем функцию, которая находит самые вероятные слова на месте маски.

In [ ]:
import torch

def probable_words(sentence, tokenizer, model, k=10):
    if sentence.count("[MASK]") != 1:
        raise ValueError("Маска должна быть ровно одна.")
    tokenization = tokenizer(sentence)["input_ids"]
    index = tokenization.index(tokenizer.mask_token_id)
    # tensor: 1 * L
    tensor = torch.LongTensor([tokenization]).to("cuda")
    with torch.no_grad():
        # logits: L * K
        logits = model(tensor)["logits"][0]
    probs = torch.softmax(logits, dim=-1)
    # top_probs: k
    top_probs, top_indexes = torch.topk(probs[index], dim=-1, k=k)
    return tokenizer.convert_ids_to_tokens(top_indexes), top_probs

In [ ]:
sentence = "The Starship prototype descended under active aerodynamic [MASK], accomplished by four vehicles."
top_tokens, top_probs = probable_words(sentence, tokenizer, model)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

In [ ]:
sentence = "I liked that [MASK] cake."
top_tokens, top_probs = probable_words(sentence, tokenizer, model)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

In [ ]:
sentence = "Yesterday, all my [MASK] seemed so far away."
top_tokens, top_probs = probable_words(sentence, tokenizer, model, k=20)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

BERT также токенизует на сабтокены, а не слова.

In [ ]:
sentence = "The Starship prototype descended under active aerodynamic control, accomplished by four vehicles."
token_indexes = tokenizer(sentence)["input_ids"]
tokens = tokenizer.convert_ids_to_tokens(token_indexes)
for index, token in zip(token_indexes, tokens):
    print(index, token)

Соответственно, чтобы найти вероятность слова aerodynamic, нужно перемножить вероятности трёх сабтокенов разбиения.

In [ ]:
def find_word_probability(sentence, word, tokenizer, model):
    """
    sentence -- предложение, содержащее ровно один символ "_", обозначающий пропуск.
    """
    if sentence.count("_") != 1:
        raise ValueError("Предложение должно содержать ровно один пропуск.")
    masked_sentence = sentence.replace("_", tokenizer.mask_token)
    masked_tokenization = tokenizer(masked_sentence)["input_ids"]
    word_tokenization = tokenizer(word, add_special_tokens=False)["input_ids"]
    word_length = len(word_tokenization)
    index = masked_tokenization.index(tokenizer.mask_token_id)
    masked_tokenization[index:index+1] = [tokenizer.mask_token_id] * word_length
    batch = torch.LongTensor([masked_tokenization]).to("cuda")
    with torch.no_grad():
        logits = model(batch)["logits"][0]
    log_probs = torch.log_softmax(logits[index:index+word_length], dim=-1).cpu().numpy()
    subtoken_log_probs = log_probs[np.arange(word_length), word_tokenization]
    total_prob = subtoken_log_probs.sum()
    return {"total_log_prob": total_prob, "subtoken_probs": np.exp(subtoken_log_probs)}

In [ ]:
import numpy as np
np.set_printoptions(precision=3)

texts = [
    "Yesterday, all my _ seemed so far away.", 
    "The Starship prototype descended under active _ control, accomplished by four vehicles."
] 
words = ["troubles", "aerodynamic"]
for text, word in zip(texts, words):
    print(text, word)
    print(find_word_probability(text, word, tokenizer, model))

На самом деле BERT неадекватно вычисляет вероятности, когда присутствует несколько масок подряд. Поэтому маскировать сабтокены следует слева направо, чтобы при вычислении вероятности последнего сабтокена все остальные были уже заполнены.

Это более точно соответствует формуле условной вероятности:
$$
p(t_1 ... t_k | \mathrm{context}) = p(t_1 | \mathrm{context}) p(t_2 | \mathrm{context}, t_1) ... p(t_k | \mathrm{context}, t_1, ..., t_{k-1})
$$

In [ ]:
def find_word_probability(sentence, word, tokenizer, model):
    """
    sentence -- предложение, содержащее ровно один символ "_", обозначающий пропуск.
    """
    if sentence.count("_") != 1:
        raise ValueError("Предложение должно содержать ровно один пропуск.")
    masked_sentence = sentence.replace("_", tokenizer.mask_token)
    masked_tokenization = tokenizer(masked_sentence)["input_ids"]
    word_tokenization = tokenizer(word, add_special_tokens=False)["input_ids"]
    word_length = len(word_tokenization)
    index = masked_tokenization.index(tokenizer.mask_token_id)
    masked_tokenization[index:index+1] = [tokenizer.mask_token_id] * word_length
    # повторяем маскированную токенизацию `word_length` раз
    # [[active MASK MASK MASK control] [active MASK MASK MASK control] [active MASK MASK MASK control]]
    batch = np.array([masked_tokenization] * word_length, dtype=int)
    # заполняем начальные сабтокены
    for prefix_length in range(1, word_length):
        '''[
            [active MASK(a) MASK MASK control] 
            [active a MASK(ero) MASK control] 
            [active a ero MASK(dynamic) control]
        ]'''
        batch[prefix_length, index:index+prefix_length] = word_tokenization[:prefix_length]
    batch = torch.LongTensor(batch).to("cuda")
    with torch.no_grad():
        logits = model(batch)["logits"]
    # в первом примере нам нужна позиция с номером index, далее index+1, ...
    print(logits.shape)
    log_probs = torch.log_softmax(
        logits[np.arange(word_length),index+np.arange(word_length)], dim=-1
    ).cpu().numpy()
    print(log_probs.shape)
    subtoken_log_probs = log_probs[np.arange(word_length), word_tokenization]
    total_prob = subtoken_log_probs.sum()
    return {"total_log_prob": total_prob, "subtoken_probs": np.exp(subtoken_log_probs)}

In [ ]:
np.set_printoptions(precision=3)

texts = [
    "Yesterday, all my _ seemed so far away.", 
    "The Starship prototype descended under active _ control, accomplished by four vehicles."
] 
words = ["troubles", "aerodynamic"]
for text, word in zip(texts, words):
    print(text, word)
    print(find_word_probability(text, word, tokenizer, model))

В целом вероятностям БЕРТа не стоит слишком доверять, когда они вычисляются для слов, состоящих из нескольких подтокенов.

### Модель для русского языка

In [ ]:
from transformers import AutoTokenizer, BertForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = BertForMaskedLM.from_pretrained("DeepPavlov/rubert-base-cased").to("cuda")

In [ ]:
sentence = "Все счастливые семьи похожи друг на друга, каждая несчастная семья несчастлива по-своему."
token_indexes = tokenizer(sentence)["input_ids"]
tokens = tokenizer.convert_ids_to_tokens(token_indexes)
for index, token in zip(token_indexes, tokens):
    print(index, token)

In [ ]:
sentence = "Что такое [MASK], это небо, плачущее небо под ногами."
top_tokens, top_probs = probable_words(sentence, tokenizer, model)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

Проверим, как модель знает грамматику и другие аспекты языка.

In [ ]:
sentence = "Казань — город-[MASK], самый красивый город России."
top_tokens, top_probs = probable_words(sentence, tokenizer, model)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

In [ ]:
sentence = "Съеденное вчера пирожное показалось ему очень [MASK] ."
top_tokens, top_probs = probable_words(sentence, tokenizer, model)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

In [ ]:
sentence = "Очень [MASK] показалось ему съеденное вчера пирожное."
top_tokens, top_probs = probable_words(sentence, tokenizer, model)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

In [ ]:
sentence = "Очень [MASK] пирожное."
top_tokens, top_probs = probable_words(sentence, tokenizer, model)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

In [ ]:
sentence = "Съеденная булка показалась ему очень [MASK], так что он купил ещё одну."
top_tokens, top_probs = probable_words(sentence, tokenizer, model)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

In [ ]:
sentence = "Съеденная булка показалась ему очень [MASK], так что он потребовал деньги назад."
top_tokens, top_probs = probable_words(sentence, tokenizer, model)
for token, prob in zip(top_tokens, top_probs):
    print(f"{token}:{prob:.3f}")

Как мы видим, модель хорошо справляется с согласованием по роду, но куда хуже с пониманием тональности и смысла текста.